<a href="https://colab.research.google.com/github/rostro36/Partisan-Responses/blob/master/8.Search%2BGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Partisan-Responses-master')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Install libraries if needed
!pip install transformers wandb neuralcoref allennlp hnswlib allennlp-models torch==1.5.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install spacy==2.1.0
!python -m spacy download en

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 703.8MB 23kB/s 
ERROR: torchvision 0.7.0+cu101 has requirement torch==1.6.0, but you'll have torch 1.5.0+cu101 which is incompatible.
  Found existing installation: torch 1.5.1
    Uninstalling torch-1.5.1:
      Successfully uninstalled torch-1.5.1


     |████████████████████████████████| 27.7MB 89kB/s 
     |████████████████████████████████| 2.1MB 52.9MB/s 
     |████████████████████████████████| 3.2MB 48.0MB/s 
     |████████████████████████████████| 92kB 13.7MB/s 
ERROR: en-core-web-sm 2.2.5 has requirement spacy>=2.2.2, but you'll have spacy 2.1.0 which is incompatible.
  Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully uninstalled blis-0.4.1
  Found existing installation: preshed 3.0.2
    Uninstalling preshed-3.0.2:
      Successfully uninstalled preshed-3.0.2
  Found existing installation: plac 1.1.3
    Uninstalling plac-1.1.3:
      Successfully uninstalled plac-1.1.3
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


     |████████████████████████████████| 11.1MB 598kB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.1.0-cp36-none-any.whl size=11074435 sha256=6e3fe258507488018416ba7842e27ad6438787836e85fd440058d2fc2887a33b
  Stored in directory: /tmp/pip-ephem-wheel-cache-u_becdh8/wheels/39/ea/3b/507f7df78be8631a7a3d7090962194cf55bc1158572c0be77f
Successfully built en-core-web-sm
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


Import all libraries

In [3]:
import numpy as np
import pandas as pd
import spacy
import pickle
import torch
import torch.nn.functional as F
from itertools import islice
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from Search import Search
from Answer import Answer
from Speech import Speech
from KnowledgeGraph import KnowledgeGraph
import neuralcoref
import re
import gc
import utils

INFO:absl:Using /tmp/tfhub_modules to cache modules.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


100%|██████████| 1345947288/1345947288 [00:20<00:00, 64755101.13B/s]


100%|██████████| 40155833/40155833 [00:01<00:00, 21226886.49B/s]


In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Load data & search

You can download "tfidf_data.pkl" from [here](https://drive.google.com/file/d/18cyDjtEo-N5DvRL6cNTSUEgtz5A1RuK9/view?usp=sharing).

In [5]:
with open("tfidf_data.pkl", "rb") as file:
    data = pickle.load(file)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [6]:
# Manual Questions (not from corpus)
questions = ["Should abortion be illegal?",
             "What do you believe about tax increases?",
             "Should same-sex marriage be legal?",
             "Is climate change real?",
             "Should immigrants be allowed to obtain citizenship?",
             "Who should be given voting rights?",
             "Should we have higher taxes for higher incomes?",
             "Should we allow death penalty?",
             "Should we have universal healthcare?",
             "Do government regulations hinder free market capitalism?",
             "What do you think about the current president?",
             "Should we reduce national debt?",
             "Should we increase spending on healthcare?",
             "Should we increase spending on education?",
             "Should every American have equal opportunities regardless of sex, age and race?",
             "Should be introduce more gun control measures?",
             "What do you think about the current president?",
             "Should Americans be free?",
             "What party do you support?",
             "What is the biggest threat to America?"
             ]

In [7]:
# create Search object
search = Search(rep_speeches=data['rep_speeches'],
                    dem_speeches=data['dem_speeches'],
                    rep_vectorizer=data['rep_vectorizer'],
                    dem_vectorizer=data['dem_vectorizer'],
                    rep_tfidf=data['rep_tfidf'],
                    dem_tfidf=data['dem_tfidf'])

NUM = 3
results = {}

# search for responses
for question in questions:
  results[question] = {}
  for party in ['R', 'D']:
    r = search.search(question, party, topk=NUM)
    results[question][party] = r.Questions.tolist()


In [9]:
df=pd.DataFrame.from_dict(results)
df.to_pickle("partisan_search_results.pkl")

Generate answers to question with GPT-2

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
for question in questions:
    print("Question: {}".format(question))
    print("GPT-2 generation: ")
    generated = tokenizer.encode(question)
    context = torch.tensor([generated])
    past = None

    for i in range(100):
        output, past = model(context, past=past)
        token = torch.argmax(output[..., -1, :])

        generated += [token.tolist()]
        context = token.unsqueeze(0)

    sequence = tokenizer.decode(generated)

    print(sequence)

Question: Should abortion be illegal?
GPT-2 generation: 
Should abortion be illegal?

The Supreme Court has ruled that abortion is illegal under the Fourteenth Amendment. The Supreme Court has ruled that abortion is illegal under the Fourteenth Amendment.

The Supreme Court has ruled that abortion is illegal under the Fourteenth Amendment.

The Supreme Court has ruled that abortion is illegal under the Fourteenth Amendment.

The Supreme Court has ruled that abortion is illegal under the Fourteenth Amendment.

The Supreme Court has ruled that abortion is illegal under the Fourteenth Amendment.
Question: What do you believe about tax increases?
GPT-2 generation: 
What do you believe about tax increases?

I think that the tax increases are going to be very good for the economy. I think that the tax increases are going to be very good for the middle class. I think that the middle class is going to be very happy. I think that the middle class is going to be very happy.

I think that the mid